In [1]:
from PIL import Image
import cv2
import numpy as np

# 1) <u>Gaussian Filter</u>

## 1.1) <u>Gaussian Filter From Scratch</u>

In [2]:
def convolution(oldimage, kernel):
    image_h = oldimage.shape[0]
    image_w = oldimage.shape[1]
    
    
    kernel_h = kernel.shape[0]
    kernel_w = kernel.shape[1]
    
    if(len(oldimage.shape) == 3):
        image_pad = np.pad(oldimage, pad_width=(
            (kernel_h // 2, kernel_h // 2),(kernel_w // 2, 
            kernel_w // 2),(0,0)), mode='constant', 
            constant_values=0).astype(np.float32)
    elif(len(oldimage.shape) == 2):
        image_pad = np.pad(oldimage, pad_width=(
            (kernel_h // 2, kernel_h // 2),(kernel_w // 2, 
            kernel_w // 2)), mode='constant', constant_values=0).astype(np.float32)
    
    
    h = kernel_h // 2
    w = kernel_w // 2
    
    image_conv = np.zeros(image_pad.shape)
    
    for i in range(h, image_pad.shape[0]-h):
        for j in range(w, image_pad.shape[1]-w):
            x = image_pad[i-h:i-h+kernel_h, j-w:j-w+kernel_w]
            x = x.flatten()*kernel.flatten()
            image_conv[i][j] = x.sum()
    h_end = -h
    w_end = -w
    
    if(h == 0):
        return image_conv[h:,w:w_end]
    if(w == 0):
        return image_conv[h:h_end,w:]
    
    return image_conv[h:h_end,w:w_end]

In [3]:
def GaussianBlurImage(image, sigma, flag = cv2.IMREAD_GRAYSCALE):
    image = cv2.imread(image, flag)
    image = np.asarray(image)
    filter_size = 2 * int(4 * sigma + 0.5) + 1
    gaussian_filter = np.zeros((filter_size, filter_size), np.float32)
    m = filter_size//2
    n = filter_size//2
    
    for x in range(-m, m+1):
        for y in range(-n, n+1):
            x1 = 2*np.pi*(sigma**2)
            x2 = np.exp(-(x**2 + y**2)/(2* sigma**2))
            gaussian_filter[x+m, y+n] = (1/x1)*x2
    
    im_filtered = np.zeros_like(image, dtype=np.float32)
    try:
        if image.shape[2] == 3:
            for c in range(3):
                im_filtered[:, :, c] = convolution(image[:, :, c], gaussian_filter)
    except Exception as e:
        im_filtered = convolution(image, gaussian_filter)
    return (im_filtered.astype(np.uint8))

In [4]:
img = Image.open("C:\\Users\\BassilOraby\\Downloads\\Lion.jpg")
img.show()
img_filtered = GaussianBlurImage("C:\\Users\\BassilOraby\\Downloads\\Lion.jpg", 4, cv2.IMREAD_UNCHANGED)
img_filtered_op = Image.fromarray(img_filtered)
img_filtered_op.show()

KeyboardInterrupt: 

## 1.2) <u>Gaussian Filter Using OpenCV Package</u>

In [4]:
img = cv2.imread("C:\\Users\\BassilOraby\\Downloads\\Lion.jpg")
# How come do we pass as parameters the kernel size and the sigma to the gaussianBlur function at the same time? isn't the sigma parameter suppossed to be 
# utilized in deciding the Kernel size ?
# Howa el X fee el sigma Horizontal wala Vertical ?
blurred_img = cv2.GaussianBlur(img, (101, 101), sigmaX=4)
cv2.imwrite("c:\\Users\\BassilOraby\\Downloads\\blurred_image.jpg", blurred_img)
output = Image.open("c:\\Users\\BassilOraby\\Downloads\\blurred_image.jpg")
output.show()
blurred_img = np.asarray(blurred_img, dtype = "uint8")
print(blurred_img)
print(img_filtered)
print(np.all(blurred_img == img_filtered))

[[[134  98  58]
  [134  98  58]
  [134  98  58]
  ...
  [151 124  98]
  [151 124  98]
  [151 124  98]]

 [[134  98  58]
  [134  98  58]
  [134  98  58]
  ...
  [151 124  98]
  [151 124  98]
  [151 124  98]]

 [[134  98  58]
  [134  98  58]
  [134  98  58]
  ...
  [151 124  98]
  [151 124  98]
  [151 124  98]]

 ...

 [[ 78 141 167]
  [ 78 141 167]
  [ 79 142 167]
  ...
  [ 30  80 114]
  [ 30  78 112]
  [ 29  78 112]]

 [[ 78 141 166]
  [ 78 141 166]
  [ 79 141 167]
  ...
  [ 30  81 115]
  [ 30  79 113]
  [ 30  78 112]]

 [[ 78 141 166]
  [ 78 141 166]
  [ 79 141 167]
  ...
  [ 31  81 115]
  [ 30  79 113]
  [ 30  78 112]]]


NameError: name 'img_filtered' is not defined

# 2) <u>Canny Edge Detector</u>

## 2.1) <u>Canny Edge Detector From Scratch</u>

In [5]:
def sobel_filters(img):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], np.float32)
    
    Ix = convolution(img, Kx)
    Iy = convolution(img, Ky)
    
    G = np.hypot(Ix, Iy)
    print(f"G before: {G}")
    G = G / G.max() * 255
    print(f"G after: {G}")
    theta = np.arctan2(Iy, Ix)
    
    return (G, theta)

In [6]:
def non_max_suppression(img, D):
    M, N = img.shape
    Z = np.zeros((M,N), dtype=np.int32)
    angle = D * 180. / np.pi
    angle[angle < 0] += 180

    
    for i in range(1,M-1):
        for j in range(1,N-1):
            try:
                q = 255
                r = 255
                
               #angle 0
                if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                    q = img[i, j+1]
                    r = img[i, j-1]
                #angle 45
                elif (22.5 <= angle[i,j] < 67.5):
                    q = img[i+1, j-1]
                    r = img[i-1, j+1]
                #angle 90
                elif (67.5 <= angle[i,j] < 112.5):
                    q = img[i+1, j]
                    r = img[i-1, j]
                #angle 135
                elif (112.5 <= angle[i,j] < 157.5):
                    q = img[i-1, j-1]
                    r = img[i+1, j+1]

                if (img[i,j] >= q) and (img[i,j] >= r):
                    Z[i,j] = img[i,j]
                else:
                    Z[i,j] = 0

            except IndexError as e:
                pass
    
    return Z

In [8]:
def threshold(img, lowThresholdRatio=0.05, highThresholdRatio=0.09):
    
    highThreshold = img.max() * highThresholdRatio;
    lowThreshold = highThreshold * lowThresholdRatio;
    
    M, N = img.shape
    res = np.zeros((M,N), dtype=np.int32)
    
    weak = 25
    strong = 255
    
    strong_i, strong_j = np.where(img >= highThreshold)
    print(f"Strong_i: {strong_i}, strong_j: {strong_j}")
    zeros_i, zeros_j = np.where(img < lowThreshold)
    
    weak_i, weak_j = np.where((img <= highThreshold) & (img >= lowThreshold))
    
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak
    
    return (res, weak, strong)

In [9]:
def hysteresis(img, weak, strong=255):
    M, N = img.shape  
    for i in range(1, M-1):
        for j in range(1, N-1):
            if (img[i,j] == weak):
                try:
                    if ((img[i+1, j-1] == strong) or (img[i+1, j] == strong) or (img[i+1, j+1] == strong)
                        or (img[i, j-1] == strong) or (img[i, j+1] == strong)
                        or (img[i-1, j-1] == strong) or (img[i-1, j] == strong) or (img[i-1, j+1] == strong)):
                        img[i, j] = strong
                    else:
                        img[i, j] = 0
                except IndexError as e:
                    pass
    return img

In [34]:
npImage = GaussianBlurImage("C:\\Users\\BassilOraby\\Downloads\\Lion.jpg", 4)
G, theta = sobel_filters(npImage)
#imagePreview = Image.fromarray(G)
#imagePreview.show()
Z = non_max_suppression(G, theta)
#imagePreview = Image.fromarray(Z)
#imagePreview.show()
threshold_img, w, s = threshold(Z, 0.01, 0.05)
#imagePreview = Image.fromarray(threshold_img)
#imagePreview.show()
final_op = hysteresis(threshold_img, w, s)
imagePreview = Image.fromarray(final_op)
imagePreview.show()

G before: [[140.00714267 149.83324064 170.31735085 ... 225.72992713 199.64969321
  188.0904038 ]
 [149.83324064  60.81118318  62.48199741 ...  81.05553652  79.19595949
  199.64969321]
 [170.31735085  62.48199741  62.22539674 ...  80.61017306  81.05553652
  225.72992713]
 ...
 [269.29537686 104.20172743 101.83319694 ...  65.80273551  64.14047084
  155.56349186]
 [236.91348632 101.83319694 103.5857133  ...  66.46803743  62.48199741
  138.46299145]
 [220.61731573 237.32677894 271.10514565 ... 163.16862443 139.71399357
  128.70120435]]
G after: [[ 61.55486445  65.87495925  74.88090425 ...  99.24333003  87.77702029
   82.69491891]
 [ 65.87495925  26.73595123  27.47053334 ...  35.63648588  34.81891323
   87.77702029]
 [ 74.88090425  27.47053334  27.35771753 ...  35.44067953  35.63648588
   99.24333003]
 ...
 [118.39710535  45.81282844  44.77149176 ...  28.93051302  28.19968977
   68.39429384]
 [104.16023967  44.77149176  45.54199464 ...  29.22301646  27.47053334
   60.87597038]
 [ 96.9955439

## 2.2) <u>Canny Edge Detector Using OpenCV

In [35]:
img_ocv = cv2.imread("C:\\Users\\BassilOraby\\Downloads\\Lion.jpg", cv2.IMREAD_GRAYSCALE)
blur = cv2.GaussianBlur(img_ocv, (9, 9), 4)
edges = cv2.Canny(blur, 0.1, 25)
cv2.imwrite("c:\\Users\\BassilOraby\\Downloads\\edges_image.jpg", edges)
#cv2.imshow("Original", img_ocv)
#cv2.imshow("Edges", edges)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

True